# Import Package & Read in data

In [1]:
import pandas as pd
df = pd.read_csv('data/SpotifyAudioFeatures2019.csv')
df.head()

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


# Data Wrangling

In [2]:
df = df.dropna()
df.isnull().sum().sum()

0

In [8]:
df_features= df.drop(columns=['artist_name', 'track_id', 'track_name'])
df_features.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


# Create Prediction Model

In [37]:
from sklearn.preprocessing import RobustScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree
scaler = RobustScaler()
X = scaler.fit_transform(df_features)
model = KDTree(X)

# Sample X For Prediction

In [45]:
import numpy as np
X_Prediction = np.array([X[0]])
X_Prediction

array([[-3.26240900e-01,  5.14925373e-01,  4.72894652e-01,
        -6.96569921e-01, -3.38636364e-04, -6.66666667e-01,
        -3.09025271e-01,  4.84390087e-02,  0.00000000e+00,
         3.91897891e+00,  1.92307692e+00,  0.00000000e+00,
        -7.29468599e-01, -2.25806452e-01]])

In [48]:
dist, ind = model.query(X_Prediction, k=6)
df_dist = pd.DataFrame(dist)
pd.DataFrame(ind).head()

,0,1,2,3,4,5
0,0,43436,51256,92348,24902,53729


In [49]:
pd.DataFrame(df_dist).head()

,0,1,2,3,4,5
0,0.0,1.144224,1.274368,1.304762,1.316577,1.331322


# Convert distance score back to song ID

In [50]:
prediction_list = []
for row in ind:
    song_id = [df.track_id[i] for i in row]
    prediction_list.append(song_id)

In [51]:
# Turn list in dataframe and export to csv
columns = ['id', '1', '2', '3', '4', '5']
final_prediction = pd.DataFrame(prediction_list, columns=columns)
final_prediction = final_prediction.drop_duplicates(subset=['id'], keep='first')
final_prediction.head()

,id,1,2,3,4,5
0,2RM4jf1Xa9zPgMGRDiht8O,1rG9U7m2wqM0AVmZeIsdtz,56GWUZPELOP1nK4hQwfVVV,3UDOPqusSZZ7NqyWyMMk50,02s1Voowwhr0qTSOrMVEXk,2ogGsvbGjJz71Ppc9mTExA


# Test Query

In [52]:
# Test query
final_prediction[final_prediction.id == "2RM4jf1Xa9zPgMGRDiht8O"].values.tolist()[0][1:]

['1rG9U7m2wqM0AVmZeIsdtz',
 '56GWUZPELOP1nK4hQwfVVV',
 '3UDOPqusSZZ7NqyWyMMk50',
 '02s1Voowwhr0qTSOrMVEXk',
 '2ogGsvbGjJz71Ppc9mTExA']